# Dependencies

In [9]:
# Dependencies
import pandas as pd
import re
import requests
import math 
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from pprint import pprint
from config import api_key, plotly_key
from yelpapi import YelpAPI

# Ideas
<ul><li>Average rating vs. category
<li>Average rating vs. price rating
<li>Number of restaurant categories per city 
<li>Review Count vs. city 

# Useful Links 
<ul><li><a href="https://alcidanalytics.com/p/geographic-heatmap-in-python">Heat maps</a>

# API Call

In [17]:
# API call to Yelp API
yelp_api = YelpAPI(api_key)
# Input string for location search
input_string = input("Search query: ")
api_call = yelp_api.search_query(location=input_string, limit=50)

# API Call to Plotly 
plotly.tools.set_credentials_file(username='nguyenkevint94', api_key=plotly_key)

# Delete hashtag to view the contents of api_call
pprint(api_call)

Search query: Brooklyn
{'businesses': [{'alias': 'peter-luger-brooklyn-2',
                 'categories': [{'alias': 'steak', 'title': 'Steakhouses'}],
                 'coordinates': {'latitude': 40.709945,
                                 'longitude': -73.962478},
                 'display_phone': '(718) 387-7400',
                 'distance': 5711.937381508804,
                 'id': '4yPqqJDJOQX69gC66YUDkA',
                 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/DnReRUkXRtsmKycQEYl0pg/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '178 Broadway',
                              'address2': '',
                              'address3': '',
                              'city': 'Brooklyn',
                              'country': 'US',
                              'display_address': ['178 Broadway',
                                                  'Brooklyn, NY 11211'],
                              'state': 'NY',
              

                              'state': 'NY',
                              'zip_code': '11238'},
                 'name': 'Olea',
                 'phone': '+17186437003',
                 'price': '$$',
                 'rating': 4.0,
                 'review_count': 799,
                 'transactions': ['pickup', 'delivery'],
                 'url': 'https://www.yelp.com/biz/olea-brooklyn-2?adjust_creative=wg4vS75UWcL_5IfcnZsmOg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=wg4vS75UWcL_5IfcnZsmOg'},
                {'alias': 'the-river-café-brooklyn-3',
                 'categories': [{'alias': 'newamerican',
                                 'title': 'American (New)'},
                                {'alias': 'wine_bars', 'title': 'Wine Bars'},
                                {'alias': 'venues',
                                 'title': 'Venues & Event Spaces'}],
                 'coordinates': {'latitude': 40.70318,
                                 'longitu

In [18]:
# Lists
business_names_list = []
categories_list = []
street_address_list = []
city_list = []
country_list = []
lat_list = []
lon_list = []
ratings_list = []
review_count_list = []
price_ratings = []

# Looping through each business in the call
for businesses in api_call["businesses"]:
    try:
        # Name
        name = businesses["name"]
        # print(f"Successfully found business name: {name}")

        # Category 
        category = businesses["categories"][0]["alias"]
        # print(f"Successfully found category: {category}")

        # Street Address
        street_address = businesses["location"]["address1"]
        # print(f"Successfully found street address: {street_address}")

        # City
        city = businesses["location"]["city"]
        # print(f"Successfully found city: {city}")

        # Country
        country = businesses["location"]["country"]
        # print(f"Successfully found country: {country}")
        
        # Latitude
        lat = businesses["coordinates"]["latitude"]
        # print(f"Successfully found latitude: {lat}")
        
        #Longitude 
        lon = businesses["coordinates"]["longitude"]
        # print(f"Successfully found longitude: {lon}")

        # Price rating
        # NOTE: Some places do not have a price rating (ie. $, $$, $$$)
        price = businesses["price"]
        # print(f"Successfully found price rating: {price}")

        # Ratings
        rating = businesses["rating"]
        # print(f"Successfully found rating: {rating}")

        # Review count
        review_count = businesses["review_count"]
        # print(f"Successfully found review counts: {review_count}")
        
        # Appends
        # Tried putting appends after each section, adding them towards the end made it work 
        # with no error since it passes those without a price rating
        business_names_list.append(name) 
        categories_list.append(category)
        street_address_list.append(street_address)
        city_list.append(city)
        country_list.append(country)
        lat_list.append(lat)
        lon_list.append(lon)
        price_ratings.append(price)
        ratings_list.append(rating)
        review_count_list.append(review_count)
        
        # print("- - - - - - - - - - - - - - - - - - - - - -")
    except Exception:
        pass

# Yelp DataFrame

In [19]:
# Dictionary for DataFrame
business_details_dict = ({"Name": business_names_list,
                          "Category": categories_list,
                          "Street": street_address_list,
                          "City": city_list,
                          "Country": country_list,
                          "Latitude": lat_list,
                          "Longitude": lon_list,
                          "Rating": ratings_list,
                          "Review Count": review_count_list,
                          "$": price_ratings})

# Dictionary to DataFrame
yelp_df = pd.DataFrame(business_details_dict)
yelp_df.head()

,Name,Category,Street,City,Country,Latitude,Longitude,Rating,Review Count,$
0,Peter Luger,steak,178 Broadway,Brooklyn,US,40.709945,-73.962478,4.0,5108,$$$$
1,Dough,donuts,448 Franklin Ave,Brooklyn,US,40.689070,-73.957078,4.5,1675,$
2,Juliana's Pizza,pizza,19 Old Fulton St,Brooklyn,US,40.702615,-73.993416,4.5,1873,$$
3,L & B Spumoni Gardens,pizza,2725 86th St,Brooklyn,US,40.594715,-73.981316,4.0,3294,$
4,Ample Hills Creamery,icecream,623 Vanderbilt Ave,Brooklyn,US,40.678557,-73.968475,4.5,1274,$$


# Sorting by Review Count

In [20]:
sorted_df_reviews = yelp_df.sort_values(by=["Review Count"], ascending=False)
sorted_df_reviews

,Name,Category,Street,City,Country,Latitude,Longitude,Rating,Review Count,$
0,Peter Luger,steak,178 Broadway,Brooklyn,US,40.709945,-73.962478,4.0,5108,$$$$
9,Grimaldi's,pizza,1 Front St,Brooklyn,US,40.702569,-73.993248,3.5,4436,$$
3,L & B Spumoni Gardens,pizza,2725 86th St,Brooklyn,US,40.594715,-73.981316,4.0,3294,$
6,Di Fara Pizza,pizza,1424 Ave J,Brooklyn,US,40.625093,-73.961531,4.0,3150,$$
41,Sea,thai,114 N 6th St,Brooklyn,US,40.718070,-73.960044,3.5,2759,$$
12,Pies-n-Thighs,southern,166 S 4th St,Brooklyn,US,40.711191,-73.961424,4.0,2655,$$
8,Roberta's,pizza,261 Moore St,Brooklyn,US,40.705044,-73.933571,4.0,2563,$$
18,Fette Sau,bbq,354 Metropolitan Ave,Brooklyn,US,40.714066,-73.956327,4.0,2176,$$
7,Bogota Latin Bistro,gluten_free,141 5th Ave,Brooklyn,US,40.678165,-73.978976,4.0,2019,$$
19,Soco,cajun,509 Myrtle Ave,Brooklyn,US,40.693866,-73.964535,4.0,1901,$$


# Category Shares

In [21]:
# Counting up number in each category
biz_categories = yelp_df.groupby("Category").count()
# Resetting the index to Category
biz_categories.reset_index("Category", inplace=True)
categories = biz_categories["Category"]

# Labels for each category to be used in Plotly 
labels = categories

# Values for each category to be used in Plotly
category_count = biz_categories["Name"]
values = category_count
values

# Setting up arguments for Plotly pie chart 
trace = go.Pie(labels=labels, values=values)

py.iplot([trace], filename= "Pie_Chart_Categories")

# Price Comparisons

In [22]:
# Grouping price ratings ($)
price_groups = yelp_df.groupby("$").count()
price_groups.reset_index("$", inplace=True)

labels = price_groups["$"]
values = price_groups["Name"]

trace = go.Pie(labels=labels, values=values)

py.iplot([trace], filename= "Pie_Chart_Price_Categories")

# Average Review Count vs. Category

In [23]:
# Grouping categories 
average_reviews = yelp_df.groupby("Category").mean()
average_reviews.reset_index("Category")

# Gathering top 5 categories with the highest number of reviews
top_five = average_reviews.nlargest(10,"Review Count")
top_five.reset_index("Category", inplace=True)
# Category names
categories = top_five["Category"]

# Category review counts
review_count = top_five["Review Count"]

# Setting up the bar chart
trace = go.Bar(
    x=categories,
    y=review_count,
    text=categories,
    marker=dict(
        color="rgb(158, 202, 225)",
        line=dict(
            color="rgb(8, 48,107)",
            width=1.5,
        )
    ),
    opacity=0.6
)

data=[trace]
layout=go.Layout(
    title="Top 10 Categories by Average Review Count",
)

# Bar chart 
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="top-10-categories-by-average-review-count")